In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import Counter
%matplotlib inline

#####
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.pipeline import make_pipeline

In [2]:
spdf = pd.read_csv("./StudentsPerformance.csv")
spdf.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/StudentsPerformance.csv'

In [ ]:
for c in spdf.columns:
    print(c,"  ",spdf[c].unique(),"\n")
spdf.columns

In [ ]:
for edu in spdf["parental level of education"].unique():
    edf = spdf[spdf["parental level of education"]==edu]
    print(edu,"   ",edf["math score"].mean())

In [ ]:
numeric = ["math score","reading score","writing score"]
categorical = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']

In [ ]:
## PREDICT CATEGORICAL VARIABLES BASED ON NUMERIC VARIABLES

for c in categorical:

    y = spdf[c]

    X = spdf[numeric]

    # encoder = make_column_transformer(
    #     (OneHotEncoder(sparse=False, handle_unknown="ignore"), 
    #      ["sex"]),
    #     remainder="passthrough",
    # )
    scaler = StandardScaler()
    model = KNeighborsClassifier(n_neighbors=5)
    pipeline = make_pipeline(scaler, model)

    scores_acc = cross_val_score(pipeline, X, y, cv=5, scoring="accuracy")
    print('Accuracy for '+c+':  '+str(scores_acc.mean()))

    # scores_prec = cross_val_score(pipeline, X, y, cv=50, scoring="precision")
    # print('Precision: '+str(scores_prec.mean()))

    # scores_rec = cross_val_score(pipeline, X, y, cv=50, scoring="recall")
    # print('Recall: '+str(scores_rec.mean()))

In [ ]:
males = spdf[spdf['gender']=='male']
females = spdf[spdf['gender']=='female']

fig, axs = plt.subplots(3,2,figsize=(5,10))
for i in range(len(numeric)):
    c = numeric[i]
    axs[i,0].boxplot(males[c])#,alpha=0.5,bins=10)
    axs[i,0].set_title('males - '+c)
    axs[i,1].boxplot(females[c])#,alpha=0.5,bins=10)
    axs[i,1].set_title('females - '+c)

In [ ]:
## PREDICT NUMERIC VARIABLES BASED ON CATEGORICAL VARIABLES

for n in numeric:

    y = spdf[n]

    X = spdf[categorical]

    # cats = [ x for x in f if X[x].dtype!=int ]
    encoder = make_column_transformer(
        (OneHotEncoder(sparse=False, handle_unknown="ignore"), 
         categorical),
        remainder="passthrough",
    )
    scaler = StandardScaler()
    model = KNeighborsRegressor(n_neighbors=5)
    pipeline = make_pipeline(encoder, scaler, model)
    
    # Calculate MSE
    
    mse = np.mean(-cross_val_score(
        pipeline, X, y, 
        cv=10, scoring="neg_mean_squared_error"
    ))
    print('MSE for '+n+':  '+str(mse))
    
    # Visualize predictions
    
    encoder.fit(X)
    X_enc = encoder.transform(X)
    scaler.fit(X_enc)
    X_sc = scaler.transform(X_enc)
    model.fit(X_sc,y)
    preds = model.predict(X_sc)
    
    clrmap = {'male':'b','female':'r'}
    plt.figure()
    plt.scatter(x=preds,y=y,c=X['gender'].map(clrmap))
    plt.xlabel('Predicted '+n+' Score')
    plt.ylabel('Actual '+n+' Score')
    